In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from sklearn import metrics
import matplotlib.dates as mdates
%matplotlib inline
import seaborn as sns

Time series + EMA

Discover users who are accessing our curriculum pages way beyond the end of their codeup time. What would the dataframe look like? Use time series method for detecting anomalies, like exponential moving average with %b.

Bonus:

Can you label students who are viewing both the web dev and data science curriculum? Can you label students by the program they are in? Can you label users by student vs. staff? What are Zach, Maggie, David, and Ryan's ids?

In [2]:
colnames=['date', 'timestamp', 'content', 'num1', 'num2',
          'ip']
df = pd.read_csv('anonymized-curriculum-access.txt', sep=r'\s(?=(?:[^"]*"[^"]*")*[^"]*$)(?![^\[]*\])',
                  engine='python',
                  header=None,
                  names=colnames)

In [3]:
df.head()

,date,timestamp,content,num1,num2,ip
0,2018-01-26,09:55:03,/,1,8.0,97.105.19.61
1,2018-01-26,09:56:02,java-ii,1,8.0,97.105.19.61
2,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61
3,2018-01-26,09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61
4,2018-01-26,09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61


In [4]:
df.shape

(399195, 6)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 399195 entries, 0 to 399194
Data columns (total 6 columns):
date         399195 non-null object
timestamp    399195 non-null object
content      399195 non-null object
num1         399195 non-null int64
num2         372441 non-null float64
ip           399195 non-null object
dtypes: float64(1), int64(1), object(4)
memory usage: 18.3+ MB


In [6]:
df.describe()

,num1,num2
count,399195.000000,372441.000000
mean,268.794667,26.912649
std,147.587312,7.846146
min,1.000000,1.000000
25%,157.000000,23.000000
50%,288.000000,27.000000
75%,392.000000,31.000000
max,543.000000,52.000000


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 399195 entries, 0 to 399194
Data columns (total 6 columns):
date         399195 non-null object
timestamp    399195 non-null object
content      399195 non-null object
num1         399195 non-null int64
num2         372441 non-null float64
ip           399195 non-null object
dtypes: float64(1), int64(1), object(4)
memory usage: 18.3+ MB


In [8]:
# combine date and timestamp, then change to datetime, make index
# combine to be able to use together when possibly searching for anomalies in time increments
df['timestamp'] = df.date +' ' + df.timestamp
df.drop(columns=('date'), inplace=True)

In [9]:
df.set_index(['timestamp'], inplace=True)

In [10]:
df.tail()

,content,num1,num2,ip
timestamp,,,,
2019-11-19 10:06:28,spring/fundamentals/controllers,445,33.0,97.105.19.58
2019-11-19 10:06:50,spring/fundamentals/views,445,33.0,97.105.19.58
2019-11-19 10:07:03,8-clustering/3-wrangle,475,34.0,97.105.19.58
2019-11-19 10:07:07,7-classification/2-acquire,475,34.0,97.105.19.58
2019-11-19 10:07:45,search/search_index.json,455,33.0,97.105.19.58


In [11]:
df.num1.unique()

array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
        79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
        92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104,
       105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117,
       118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130,
       131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143,
       144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156,
       157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169,
       170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 18

In [12]:
# look through content to look at path, potentially separate out path?
df['content'].nunique()

1641

In [13]:
# split content string on / to separate into new columns
df['content'] = df.content.str.split('/')

In [14]:
df.content

timestamp
2018-01-26 09:55:03                                                 [, ]
2018-01-26 09:56:02                                            [java-ii]
2018-01-26 09:56:05               [java-ii, object-oriented-programming]
2018-01-26 09:56:06                [slides, object_oriented_programming]
2018-01-26 09:56:24                         [javascript-i, conditionals]
2018-01-26 09:56:41                                [javascript-i, loops]
2018-01-26 09:56:46                         [javascript-i, conditionals]
2018-01-26 09:56:48                            [javascript-i, functions]
2018-01-26 09:56:59                                [javascript-i, loops]
2018-01-26 09:58:26                            [javascript-i, functions]
2018-01-26 09:59:22                          [mkdocs, search_index.json]
2018-01-26 10:00:02    [javascript-i, introduction, working-with-data...
2018-01-26 10:00:37                                                 [, ]
2018-01-26 10:00:39                      

In [15]:
# create new columns to separate out path

In [16]:
df.content.value_counts()

[, ]                                                                                 20564
[javascript-i]                                                                        8140
[search, search_index.json]                                                           7005
[toc]                                                                                 6679
[html-css]                                                                            6143
[java-iii]                                                                            6003
[spring]                                                                              5761
[jquery]                                                                              5501
[java-ii]                                                                             5386
[mysql]                                                                               5103
[java-i]                                                                              5039

In [17]:
df.ip.nunique()

2585

In [18]:
df.ip.value_counts()

97.105.19.58       227220
97.105.19.61        61662
192.171.117.210      8127
70.122.40.62         1338
66.69.108.117        1199
173.174.211.206      1019
108.65.244.91        1007
170.248.173.245       997
67.11.157.190         972
66.69.62.219          934
70.118.8.243          876
72.181.104.64         865
170.248.173.247       828
216.1.153.162         823
67.11.215.97          818
76.185.131.226        793
173.173.113.51        779
67.11.239.2           777
99.61.24.168          765
67.10.174.56          739
72.181.98.75          681
45.23.250.16          663
67.10.161.55          651
136.50.18.71          631
76.204.31.63          618
4.16.216.161          610
72.181.110.80         607
170.248.173.246       590
67.10.166.10          578
70.121.163.5          577
                    ...  
72.181.37.22            1
99.203.26.219           1
70.114.21.86            1
99.203.27.185           1
165.91.12.128           1
174.207.16.237          1
103.208.220.141         1
131.194.172.

In [19]:
df.ip.nunique()

2585

In [20]:
df.head()

,content,num1,num2,ip
timestamp,,,,
2018-01-26 09:55:03,"[, ]",1,8.0,97.105.19.61
2018-01-26 09:56:02,[java-ii],1,8.0,97.105.19.61
2018-01-26 09:56:05,"[java-ii, object-oriented-programming]",1,8.0,97.105.19.61
2018-01-26 09:56:06,"[slides, object_oriented_programming]",1,8.0,97.105.19.61
2018-01-26 09:56:24,"[javascript-i, conditionals]",2,22.0,97.105.19.61


In [21]:
df.num1.nunique()

543

In [22]:
df.num1.head()

timestamp
2018-01-26 09:55:03    1
2018-01-26 09:56:02    1
2018-01-26 09:56:05    1
2018-01-26 09:56:06    1
2018-01-26 09:56:24    2
Name: num1, dtype: int64

In [23]:
df.num1.unique()

array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
        79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
        92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104,
       105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117,
       118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130,
       131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143,
       144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156,
       157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169,
       170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 18

In [24]:
# cohort number = num2 column
df.num2.value_counts()

29.0    35969
24.0    35039
33.0    34433
22.0    28875
23.0    28056
32.0    26801
26.0    26760
25.0    25233
31.0    22665
28.0    20677
27.0    20198
34.0    15519
51.0    10835
14.0     9069
1.0      8877
21.0     7181
17.0     3792
52.0     2896
13.0     2610
8.0      1671
18.0     1603
19.0     1142
16.0      740
15.0      691
7.0       461
12.0      270
11.0      204
2.0        93
6.0        72
9.0         5
4.0         4
Name: num2, dtype: int64

In [25]:
# bayes ds cohort = 34.0
df[df.num2 == 34.0]

,content,num1,num2,ip
timestamp,,,,
2019-08-20 09:39:58,"[, ]",466,34.0,97.105.19.58
2019-08-20 09:39:59,"[, ]",467,34.0,97.105.19.58
2019-08-20 09:39:59,"[, ]",468,34.0,97.105.19.58
2019-08-20 09:40:02,"[, ]",469,34.0,97.105.19.58
2019-08-20 09:40:08,"[, ]",470,34.0,97.105.19.58
2019-08-20 09:40:15,"[, ]",471,34.0,97.105.19.58
2019-08-20 09:40:15,"[, ]",472,34.0,97.105.19.58
2019-08-20 09:40:17,"[, ]",473,34.0,97.105.19.58
2019-08-20 09:40:18,"[, ]",474,34.0,97.105.19.58


In [26]:
df.num2.nunique()

31

In [27]:
df.num2.unique()

array([ 8., 22., 21.,  1., 16., 18., 19., nan, 13., 14., 15.,  7.,  4.,
       12., 17., 23.,  2.,  9., 11., 24., 25., 26.,  6., 27., 28., 29.,
       31., 32., 33., 34., 51., 52.])

In [28]:
# num1 is student number 
df[df.num1 == 475]

,content,num1,num2,ip
timestamp,,,,
2019-08-20 09:40:19,"[, ]",475,34.0,97.105.19.58
2019-08-20 10:46:35,"[1-fundamentals, 1.1-intro-to-data-science]",475,34.0,97.105.19.58
2019-08-20 10:46:36,"[1-fundamentals, modern-data-scientist.jpg]",475,34.0,97.105.19.58
2019-08-20 10:46:36,"[1-fundamentals, AI-ML-DL-timeline.jpg]",475,34.0,97.105.19.58
2019-08-20 10:50:11,"[1-fundamentals, 1.1-intro-to-data-science]",475,34.0,97.105.19.58
2019-08-20 10:50:12,"[1-fundamentals, modern-data-scientist.jpg]",475,34.0,97.105.19.58
2019-08-20 10:50:12,"[1-fundamentals, AI-ML-DL-timeline.jpg]",475,34.0,97.105.19.58
2019-08-21 09:20:11,"[, ]",475,34.0,97.105.19.58
2019-08-21 09:20:52,"[1-fundamentals, 1.1-intro-to-data-science]",475,34.0,97.105.19.58


In [29]:
(df.num1 == 475).sum()

1316

In [30]:
#plt.plot(df.num1 == 475)

In [31]:
df.head()

,content,num1,num2,ip
timestamp,,,,
2018-01-26 09:55:03,"[, ]",1,8.0,97.105.19.61
2018-01-26 09:56:02,[java-ii],1,8.0,97.105.19.61
2018-01-26 09:56:05,"[java-ii, object-oriented-programming]",1,8.0,97.105.19.61
2018-01-26 09:56:06,"[slides, object_oriented_programming]",1,8.0,97.105.19.61
2018-01-26 09:56:24,"[javascript-i, conditionals]",2,22.0,97.105.19.61


In [33]:
df.content.value_counts()

[, ]                                                                                 20564
[javascript-i]                                                                        8140
[search, search_index.json]                                                           7005
[toc]                                                                                 6679
[html-css]                                                                            6143
[java-iii]                                                                            6003
[spring]                                                                              5761
[jquery]                                                                              5501
[java-ii]                                                                             5386
[mysql]                                                                               5103
[java-i]                                                                              5039

In [41]:
df.index

Index(['2018-01-26 09:55:03', '2018-01-26 09:56:02', '2018-01-26 09:56:05',
       '2018-01-26 09:56:06', '2018-01-26 09:56:24', '2018-01-26 09:56:41',
       '2018-01-26 09:56:46', '2018-01-26 09:56:48', '2018-01-26 09:56:59',
       '2018-01-26 09:58:26',
       ...
       '2019-11-19 10:05:00', '2019-11-19 10:05:53', '2019-11-19 10:06:13',
       '2019-11-19 10:06:21', '2019-11-19 10:06:26', '2019-11-19 10:06:28',
       '2019-11-19 10:06:50', '2019-11-19 10:07:03', '2019-11-19 10:07:07',
       '2019-11-19 10:07:45'],
      dtype='object', name='timestamp', length=399195)

In [ ]:
# start date 2018-01-26
# end date 2019-11-19

In [47]:
len(df.content.value_counts())

1641

In [48]:
df.shape

(399195, 4)

In [49]:
df.content.value_counts()

[, ]                                                                                 20564
[javascript-i]                                                                        8140
[search, search_index.json]                                                           7005
[toc]                                                                                 6679
[html-css]                                                                            6143
[java-iii]                                                                            6003
[spring]                                                                              5761
[jquery]                                                                              5501
[java-ii]                                                                             5386
[mysql]                                                                               5103
[java-i]                                                                              5039

In [54]:
from scipy.stats import entropy

In [55]:
def compute_entropy(series):
    counts = series.value_counts()
    if len(counts)==1:
        ent = 0
    else:
        value, counts = np.unique(series, return_counts=True)
        ent = entropy(counts, base=None)
    return ent

In [56]:
compute_entropy(df.content)

5.421441033159645